In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

## Xavier and He Initialization

In [ ]:
tf.keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 3s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 3s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 3s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 3s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 3s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 3s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 3s - loss: 0.3485 - accuracy: 0.9021 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 3s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 3s - loss: 0.3251 - accuracy: 0.9078 - val_loss: 0.3016 - 

In [ ]:
model.save("full_mnist_model.h5")

In [ ]:
load_full_mnist_model = tf.keras.models.load_model("full_mnist_model.h5")

In [ ]:
load_full_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [ ]:
np.where(y_train % 2 == 0, 1 , 0)

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
y_train[0], y_train[-1] 

(7, 8)

In [ ]:
def update_even_odd_labels(labels):
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label % 2 == 0, 1 , 0)
    return labels

In [ ]:
y_train_bin, y_valid_bin, y_test_bin = update_even_odd_labels([y_train, y_valid, y_test])

In [ ]:
# Check trainable or not:

for layer in load_full_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: True
dense_1: True
leaky_re_lu: True
dense_2: True
leaky_re_lu_1: True
dense_3: True


In [ ]:
# make lower layer UN-trainable except the last layer:

for layer in load_full_mnist_model.layers[:-1]:
    layer.trainable = False
    print(f"Now {layer.name}: {layer.trainable}")

Now flatten: False
Now dense_1: False
Now leaky_re_lu: False
Now dense_2: False
Now leaky_re_lu_1: False


In [ ]:
new_model = tf.keras.models.Sequential(load_full_mnist_model.layers[:-1])
new_model.add(
    tf.keras.layers.Dense(1, activation="sigmoid")
)

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 265,701
Trainable params: 101
Non-trainable params: 265,600
______________________________________________

In [ ]:
new_model.get_config()

{'name': 'sequential_1',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'flatten_input'}},
  {'class_name': 'Flatten',
   'config': {'name': 'flatten',
    'trainable': False,
    'batch_input_shape': (None, 28, 28),
    'dtype': 'float32',
    'data_format': 'channels_last'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': False,
    'dtype': 'float32',
    'units': 300,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'HeNormal', 'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'LeakyReLU',
   'config': {'name': 'leaky_re_lu',
    'trainable': False,
    'dtype': 'float32',
    'al

In [ ]:
# Check trainable or not:

for layer in new_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: False
dense_1: False
leaky_re_lu: False
dense_2: False
leaky_re_lu_1: False
dense_4: True


In [ ]:
new_model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
new_history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 2s - loss: 0.4630 - accuracy: 0.7917 - val_loss: 0.3569 - val_accuracy: 0.8564
Epoch 2/10
1719/1719 - 2s - loss: 0.3552 - accuracy: 0.8486 - val_loss: 0.3265 - val_accuracy: 0.8626
Epoch 3/10
1719/1719 - 2s - loss: 0.3365 - accuracy: 0.8554 - val_loss: 0.3127 - val_accuracy: 0.8686
Epoch 4/10
1719/1719 - 2s - loss: 0.3262 - accuracy: 0.8603 - val_loss: 0.3041 - val_accuracy: 0.8712
Epoch 5/10
1719/1719 - 2s - loss: 0.3191 - accuracy: 0.8631 - val_loss: 0.2979 - val_accuracy: 0.8740
Epoch 6/10
1719/1719 - 2s - loss: 0.3136 - accuracy: 0.8659 - val_loss: 0.2929 - val_accuracy: 0.8756
Epoch 7/10
1719/1719 - 2s - loss: 0.3092 - accuracy: 0.8683 - val_loss: 0.2889 - val_accuracy: 0.8800
Epoch 8/10
1719/1719 - 2s - loss: 0.3055 - accuracy: 0.8699 - val_loss: 0.2856 - val_accuracy: 0.8828
Epoch 9/10
1719/1719 - 2s - loss: 0.3022 - accuracy: 0.8721 - val_loss: 0.2829 - val_accuracy: 0.8844
Epoch 10/10
1719/1719 - 2s - loss: 0.2994 - accuracy: 0.8733 - val_loss: 0.2798 - 

In [ ]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 0s 1ms/step - loss: 0.2918 - accuracy: 0.8764


[0.291761577129364, 0.8763999938964844]

In [ ]:
X_new = X_test[:3]
y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [ ]:
new_model.predict(X_new)

array([[0.02084461],
       [0.96824026],
       [0.11971378]], dtype=float32)